## 1) Importing

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# 2) Device

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 3) Transform

In [3]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# 4) Dataset

# - Train The Data

In [4]:
train_data = datasets.FashionMNIST(
    root="./data",
    train=True,
    transform=transform,
    download=True
)

100%|██████████| 26.4M/26.4M [00:01<00:00, 16.1MB/s]
100%|██████████| 29.5k/29.5k [00:00<00:00, 273kB/s]
100%|██████████| 4.42M/4.42M [00:00<00:00, 4.90MB/s]
100%|██████████| 5.15k/5.15k [00:00<00:00, 13.1MB/s]


# - Test The Data

In [5]:
test_data = datasets.FashionMNIST(
    root='./data',
    train=False,
    transform=transform,
    download=True
)

# - Load The Data

In [6]:
train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = DataLoader(test_data, batch_size=64, shuffle=False)

# 5) CNN Model

In [10]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()

        self.conv1 = nn.Conv2d(1, 32, kernel_size=3)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3)
        self.pool = nn.MaxPool2d(2, 2)

        self.fc1 = nn.Linear(64 * 5 * 5, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))  # 28→26→13
        x = self.pool(torch.relu(self.conv2(x)))  # 13→11→5
        x = x.view(x.size(0), -1)                 # Flatten
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model = CNN().to(device)

# 6) Loss & Optimizer

In [11]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 8) Training

In [12]:
epochs = 5
for epoch in range(epochs):
    running_loss = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{epochs}], Loss: {running_loss:.4f}")

Epoch [1/5], Loss: 437.9397
Epoch [2/5], Loss: 289.0833
Epoch [3/5], Loss: 245.4000
Epoch [4/5], Loss: 217.3613
Epoch [5/5], Loss: 191.7445


In [13]:
print("Training Complete")

Training Complete
